In [ ]:
import numpy as np
from psycopg2 import connect
import pandas as pd
import pm4py
import numpy as np
import pandasql as ps
from pm4py.objects.conversion.log import converter as log_converter
from scipy.stats import variation
#connect to postgres mimic database
con = connect(dbname="postgres", host="127.0.0.1", user="postgres", password="1234")
con.set_client_encoding('utf8')
cursor = con.cursor()
import matplotlib.cm as cm
import matplotlib
import sklearn.preprocessing as sk
import re

In [ ]:
cursor.execute("SELECT * from mimic_hosp.drgcodes")
drgs = cursor.fetchall()
cols = list(map(lambda x: x[0], cursor.description))
drgs = pd.DataFrame(drgs, columns=cols)

In [ ]:
pd.set_option("display.max_rows", 500)

In [ ]:
pd.set_option("display.max_columns", 500)

In [ ]:
cursor.execute('SELECT * FROM mimic_hosp.diagnoses_icd')
icds = cursor.fetchall()
cols = list(map(lambda x: x[0], cursor.description))
icds = pd.DataFrame(icds, columns=cols)

In [ ]:
cursor.execute('SELECT * FROM mimic_hosp.d_icd_diagnoses')
desc_icd = cursor.fetchall()
cols = list(map(lambda x: x[0], cursor.description))
desc_icd = pd.DataFrame(desc_icd, columns=cols)

In [ ]:
icds = icds.merge(desc_icd, on=["icd_code", "icd_version"], how="inner")

In [ ]:
icds["icd_code"] = icds["icd_code"].str.replace(" ", "")

In [ ]:
icd9 = icds.loc[icds["icd_version"] == 9]

In [ ]:
icd10 = icds.loc[icds["icd_version"] == 10]

In [ ]:
hf = icds.loc[icds["icd_code"].str.contains("N17")]

In [ ]:
hf = hf.loc[hf["seq_num"] <= 3]

In [ ]:
hf_9 = icds.loc[(icds["icd_code"].str.contains("5849") | (icds["icd_code"].str.contains("5845")) | (icds["icd_code"].str.contains("5848")) |
        (icds["icd_code"].str.contains("5847"))) & (icds["icd_version"]==9)]

In [ ]:
hf_9 = hf_9.loc[hf_9["seq_num"] <= 3]

In [ ]:
hf = pd.concat([hf, hf_9])

In [ ]:
hf = hf.loc[hf["long_title"] != 'Other specified aftercare following surgery']

In [ ]:
hf = hf.reset_index()

In [ ]:
hf = hf.drop("index", axis=1)

In [ ]:
hf = hf.drop_duplicates("hadm_id")

In [ ]:
hadms = list(hf["hadm_id"])

In [ ]:
cursor.execute("SELECT * from mimic_core.transfers")
transfers = cursor.fetchall()
cols = list(map(lambda x: x[0], cursor.description))
transfers = pd.DataFrame(transfers, columns=cols)

In [ ]:
transfers = transfers.sort_values(["hadm_id", "intime"])

In [ ]:
transfers = transfers.loc[~transfers["hadm_id"].isna()]

In [ ]:
transfers.reset_index(inplace=True)

In [ ]:
transfers.drop("index", axis=1, inplace=True)

In [ ]:
hf_t = transfers.loc[transfers["hadm_id"].isin(hadms)]

In [ ]:
hf_t.loc[hf_t["outtime"].isna(), "outtime"] = hf_t["intime"] + pd.Timedelta(seconds=1)

In [ ]:
hf_t.reset_index(inplace=True)
hf_t.drop("index", axis=1, inplace=True)

In [ ]:
hf_t.loc[(hf_t["careunit"].isna()) & (hf_t["eventtype"] == "discharge"), "careunit"] = "Discharged"

In [ ]:
hf_t.loc[(hf_t["careunit"].isna()), "careunit"] = "Unknown"

In [ ]:
icu = list(hf_t.loc[hf_t["careunit"].str.contains("Care Unit")]["hadm_id"])

In [ ]:
first_dep = hf_t.groupby("hadm_id").first()

In [ ]:
hadms_f_d = list(first_dep.loc[first_dep["careunit"] == "Emergency Department"].reset_index()["hadm_id"])

In [ ]:
hf_t_ed = hf_t.loc[hf_t["hadm_id"].isin(hadms_f_d)]

In [ ]:
hf_t_ed = hf_t_ed.loc[hf_t["hadm_id"].isin(icu)]

In [ ]:
hf_icu = hf_t_ed

In [ ]:
hf_icu.loc[(hf_icu["careunit"]=="Emergency Department"), "department"] = "Emergency Department"

In [ ]:
hf_icu.loc[(hf_icu["careunit"].str.contains("CCU")) | (hf_icu["careunit"].str.contains("CVICU")), "department"] = "Cardiac ICU"

In [ ]:
hf_icu.loc[(hf_icu["careunit"].str.contains("SICU")), "department"] = "Surgical ICU"

In [ ]:
hf_icu.loc[(hf_icu["careunit"].str.contains("MICU")) & (~hf_icu["careunit"].str.contains("SICU")), "department"] = "Medical ICU"

In [ ]:
hf_icu.loc[(hf_icu["careunit"]== "Medicine"), "department"] = "Medicine"

In [ ]:
hf_icu.loc[(hf_icu["careunit"]=="Medicine/Cardiology") | (hf_icu["careunit"]=="Medicine/Cardiology Intermediate"), "department"] = "Cardiology"

In [ ]:
hf_icu.loc[(hf_icu["careunit"]=="Cardiology Surgery Intermediate") | (hf_icu["careunit"]=="Cardiology Surger") | (hf_icu["careunit"].str.contains("Med/Surg")), "department"] = "Surgery"

In [ ]:
hf_icu.loc[(hf_icu["careunit"]== "Discharged"), "department"] = "Discharged"

In [ ]:
to_remove = list(hf_icu.loc[hf_icu["department"].isna()]["hadm_id"].unique())

In [ ]:
hf_icu_clean = hf_icu.loc[~hf_icu["hadm_id"].isin(to_remove)]

In [ ]:
hf_icu_clean["transfer_duration"] = hf_icu_clean["outtime"] - hf_icu_clean["intime"]

In [ ]:
hf_icu_clean = hf_icu_clean.reset_index().sort_values(["hadm_id", "intime"]).drop("index", axis=1)

In [ ]:
newdf = pd.DataFrame(data=None, columns=hf_icu_clean.columns)

In [ ]:
hadms = hf_icu_clean["hadm_id"].unique()

In [ ]:
d = {'subject_id':'first', 'hadm_id':'first', 'transfer_id':'first', 'eventtype':'first', 'careunit':'first', 'intime':'min', 'outtime':'max'}
for i in hadms:
    hadm = hf_icu_clean.loc[hf_icu_clean["hadm_id"] == i]
    consecutive_array = (hadm.department != hadm.department.shift()).cumsum().values
    new_hadm = hadm.groupby([consecutive_array, 'department']).agg(d).reset_index(level=1)
    newdf = pd.concat([newdf, new_hadm])
    
    

In [ ]:
hf_consecutive = newdf

In [ ]:
hadms = hf_consecutive["hadm_id"].unique()

In [ ]:
hf_consecutive = hf_consecutive.reset_index().drop("index", axis=1)

In [ ]:
hf_consecutive = hf_consecutive.reset_index()

In [ ]:
icu_flag = 0
for i in hadms:
    hadm = hf_consecutive.loc[hf_consecutive["hadm_id"] == i]
    for index, row in hadm.iterrows():
        r = row
        if("ICU" in row["department"]):
            icu_flag = 1
        if((row["department"] != "Emergency Department") & (row["department"] != "Discharged") & ("ICU" not in row["department"])):
            if(icu_flag == 1):
                hf_consecutive.at[row["index"], "department"] = "Post-ICU " + row["department"] 
            else:
                hf_consecutive.at[row["index"], "department"] = "Pre-ICU " + row["department"] 
    icu_flag = 0

In [ ]:
hf_consecutive["transfer_duration"] = hf_consecutive["outtime"] - hf_consecutive["intime"]

In [ ]:
hadms = list(hf_consecutive["hadm_id"].unique())

In [ ]:
#consider only patients which have been in the ICU once
number_of_icu_visits = hf_consecutive.loc[hf_consecutive["department"].str.contains('Cardiac ICU|Medical ICU|Surgical ICU')].groupby("hadm_id").count().reset_index()

In [ ]:
to_remove = list(number_of_icu_visits.loc[number_of_icu_visits["subject_id"] > 1]["hadm_id"])

In [ ]:
hf_consecutive = hf_consecutive.loc[~hf_consecutive["hadm_id"].isin(to_remove)]

In [ ]:
hf_lab = hf_consecutive

In [ ]:
cursor.execute('SELECT * FROM mimic_hosp.d_labitems')
lab_d = cursor.fetchall()
cols = list(map(lambda x: x[0], cursor.description))
lab_d = pd.DataFrame(lab_d, columns=cols)   

In [ ]:
def build_sql_query(table_module: str, table_name: str, id_type: str) -> str:
    """Generates sql query"""
    sql_query = 'select ' + table_module + '.'+ table_name +'.* \
                       from ' + table_module + '.'+ table_name +' join (values {0}) \
                       as to_join('+ id_type + ') \
                       ON ' + table_module + '.'+ table_name +'.' + id_type + ' \
                       = to_join.' + id_type
    return sql_query

In [ ]:
def prepare_id_list_for_sql(id_list) -> str:
    """Prepares a list of ids for the sql statement"""
    id_list = [str(i) for i in id_list]
    sql_list = '), ('.join(['', *id_list, ''])
    sql_list = sql_list[3:]
    sql_list = sql_list[:len(sql_list)-3]
    return sql_list

In [ ]:
sql_id_list = prepare_id_list_for_sql(hadms)
sql_query = build_sql_query("mimic_hosp", "labevents", "hadm_id")
cursor.execute(sql_query.format(sql_id_list))
hf_lab = cursor.fetchall()
cols = list(map(lambda x: x[0], cursor.description))
hf_lab = pd.DataFrame(hf_lab, columns=cols)

In [ ]:
hf_lab = hf_lab.merge(lab_d, on="itemid", how="inner")

In [ ]:
count = hf_lab.groupby("label").count().sort_values("subject_id", ascending=False).head(30)

In [ ]:
lab_labels = list(count.reset_index()["label"])

In [ ]:
hf_lab = hf_lab.sort_values(["subject_id", "hadm_id", "charttime"])

In [ ]:
to_merge = hf_consecutive[["hadm_id", "intime", "outtime", "department"]]

In [ ]:
hf_consecutive.to_csv("hf_consecutive_EAA.csv")

In [ ]:
#lab event to department
sqlcode = '''
select *
from hf_lab
inner join to_merge on hf_lab.hadm_id=to_merge.hadm_id
where hf_lab.charttime >= to_merge.intime and hf_lab.charttime <= to_merge.outtime 

'''

newdf = ps.sqldf(sqlcode,locals())
newdf = newdf.loc[:,~newdf.columns.duplicated()]
newdf = newdf.sort_values(["subject_id", "hadm_id", "charttime"])
newdf = newdf.reset_index()
newdf = newdf.drop("index", axis=1)

In [ ]:
#Make last lab value the discharge lab value

In [ ]:
discharge_info = newdf.groupby(["subject_id", "hadm_id"]).last().reset_index()[["subject_id", "hadm_id", "charttime"]]

In [ ]:
# the dictionary to pass to pandas dataframe
d = {}

# a counter to use to add entries to "dict"
i = 0 

for index, row in discharge_info.iterrows():
    labs = newdf.loc[(newdf["hadm_id"] == row["hadm_id"]) & (newdf["charttime"] == row["charttime"])]
    labs["department"] = "Discharged"
    d[i] = labs
    i = i + 1


In [ ]:
to_concat = pd.DataFrame()
for i in range(0, len(d)-1):
    to_concat = pd.concat([to_concat, d[i]])

In [ ]:
newdf = pd.concat([newdf, to_concat])

In [ ]:
#remove lab values from emergency department as they are under representative, most values n < 100
newdf = newdf.loc[newdf["department"] != "Emergency Department"]

In [ ]:
x = newdf.groupby(["subject_id","hadm_id", "department", "label", "intime", "outtime"]).agg({"valuenum":"mean"}).reset_index()

In [ ]:
lab_df = x.loc[x["label"].isin(lab_labels)]

In [ ]:
lab_df = lab_df.loc[~lab_df["valuenum"].isna()]

In [ ]:
to_merge = newdf.groupby(["hadm_id", "label"]).first()

to_merge = to_merge.reset_index()

to_merge = to_merge[["hadm_id", "label", "ref_range_lower","ref_range_upper"]]

In [ ]:
#put ref here
x = x.merge(to_merge, on=["hadm_id", "label"], how="left")

In [ ]:
lab_df = x.loc[x["label"].isin(lab_labels)]

In [ ]:
lab_df.loc[lab_df["valuenum"] < lab_df["ref_range_lower"], "abnormal"] = "abnormal low"
lab_df.loc[lab_df["valuenum"] > lab_df["ref_range_upper"], "abnormal"] = "abnormal high"
lab_df.loc[(lab_df["valuenum"] >= lab_df["ref_range_lower"]) & (lab_df["valuenum"] <= lab_df["ref_range_upper"]), "abnormal"] = "normal"

In [ ]:
lab_df = lab_df.loc[~lab_df["valuenum"].isna()]

In [ ]:
lab_df.drop(["ref_range_lower", "ref_range_upper"], axis=1, inplace=True)

In [ ]:
#merge on hadm_id, department, intime, outtime
test = lab_df.loc[lab_df["department"] == "Discharged"]

In [ ]:
lab_df = lab_df.reset_index()
lab_df.drop("index", axis=1, inplace=True)

In [ ]:
#lab values of Discharged do not have the correct timestamp yet and cannot be merged!
deps = hf_consecutive.loc[hf_consecutive["department"] == "Discharged"][["subject_id", "hadm_id", "intime", "outtime", "department"]]
lab_df.loc[lab_df["department"] == "Discharged"]
for index, row in lab_df.iterrows():
    if (row["department"] == "Discharged"):
        time = deps.loc[deps["hadm_id"] == row["hadm_id"]]
        time = time.reset_index()
        lab_df.loc[index, "intime"] = time["intime"][0]
        lab_df.loc[index, "outtime"] = time["outtime"][0]
        

In [ ]:
lab_df["intime"] = lab_df["intime"].apply(lambda x: pd.to_datetime(x))

In [ ]:
lab_df["outtime"] = lab_df["outtime"].apply(lambda x: pd.to_datetime(x))

In [ ]:
len(lab_df.loc[lab_df["department"] == "Discharged"]["hadm_id"].unique())

In [ ]:
#make log ready for process mining --> for each lab value a separate column
lab_pm = hf_consecutive[["subject_id", "hadm_id", "transfer_id", "eventtype", "careunit", "intime", "outtime", "department"]]
for label in lab_labels:
    df_single = lab_df.loc[lab_df["label"] == label]
    df_single.rename({"valuenum":label, "abnormal":"Abnormal "+ label}, axis=1, inplace=True)
    df_single.drop("label", axis=1, inplace=True)
    lab_pm = lab_pm.merge(df_single, on=["subject_id", "hadm_id", "intime", "outtime", "department"], how="left")

In [ ]:
cursor.execute('SELECT * FROM mimic_core.admissions where hadm_id = any(%s)', [hadms])
adms = cursor.fetchall()
cols = list(map(lambda x: x[0], cursor.description))
adms = pd.DataFrame(adms, columns=cols)

In [ ]:
ed = adms[["hadm_id", "admission_location", "insurance", "ethnicity", "marital_status"]]

In [ ]:
discharge = adms[["hadm_id", "discharge_location"]]

In [ ]:
ed["department"] = "Emergency Department"

In [ ]:
discharge["department"] = "Discharged"

In [ ]:
lab_demo = lab_pm.merge(ed, on=["hadm_id", "department"], how="left")

In [ ]:
lab_demo = lab_demo.merge(discharge, on=["hadm_id", "department"], how="left")

In [ ]:
cursor.execute('SELECT * FROM mimic_core.patients')
patients = cursor.fetchall()
cols = list(map(lambda x: x[0], cursor.description))
patients = pd.DataFrame(patients, columns=cols)

In [ ]:
patients["department"] = "Emergency Department"

In [ ]:
patients = patients[["subject_id", "department", "anchor_age"]]

In [ ]:
lab_demo = lab_demo.merge(patients, on=["subject_id", "department"], how="left")

In [ ]:
final_pm = lab_demo

In [ ]:
final_pm = final_pm.rename({"anchor_age":"age"}, axis=1)

In [ ]:
final_pm.drop("careunit", axis=1, inplace=True)

In [ ]:
final_pm.drop("eventtype", axis=1, inplace=True)

In [ ]:
final_pm = final_pm.drop(["Specimen Type", "Abnormal Specimen Type", "Base Excess", "Abnormal Base Excess"], axis=1)

In [ ]:
final_pm = final_pm.rename({"Calcium, Total":"Calcium", "Abnormal Calcium, Total":"Abnormal Calcium"}, axis=1)

In [ ]:
final_pm

In [ ]:
hadms = list(final_pm["hadm_id"].unique())

In [ ]:
cursor.execute('SELECT * FROM mimic_ed.edstays where hadm_id = any(%s)', [hadms])
ed_stay = cursor.fetchall()
cols = list(map(lambda x: x[0], cursor.description))
ed_stay = pd.DataFrame(ed_stay, columns=cols)

In [ ]:
ed_stay = ed_stay[["subject_id", "hadm_id", "stay_id"]]

In [ ]:
ed_stay.rename({"stay_id": "ed_stay_id"}, axis=1, inplace=True)

In [ ]:
stays = list(ed_stay["ed_stay_id"])

In [ ]:
cursor.execute('SELECT * FROM mimic_ed.triage where stay_id = any(%s)', [stays])
triage = cursor.fetchall()
cols = list(map(lambda x: x[0], cursor.description))
triage = pd.DataFrame(triage, columns=cols)

In [ ]:
to_merge = triage[["stay_id", "temperature", "heartrate", "resprate", "o2sat", "sbp", "dbp", "pain", "acuity", "chiefcomplaint"]]

In [ ]:
x = final_pm.merge(ed_stay, on=["hadm_id", "subject_id"], how="left")

In [ ]:
x.loc[(~x["ed_stay_id"].isna()) & (x["department"] != "Emergency Department"), "ed_stay_id"] = np.nan

In [ ]:
final = x.merge(to_merge, left_on="ed_stay_id", right_on="stay_id", how="left")

In [ ]:
final_pm = final.drop_duplicates(["hadm_id", "department"])

In [ ]:
final_pm.drop("chiefcomplaint",axis=1, inplace=True)

In [ ]:
final_pm.to_csv("Kidney_Log.csv")